## How to use pyctcdecode when working with a Huggingface model

In [ ]:
# install huggingface transformers
!pip install transformers==4.11.2

In [2]:
# load pretrained model
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
asr_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
asr_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# get a single audio file
!wget https://dldata-public.s3.us-east-2.amazonaws.com/1919-142785-0028.wav

In [3]:
# transcribe audio to logits over characters
import soundfile as sf
arr, _ = sf.read('1919-142785-0028.wav')
input_values = asr_processor(arr, return_tensors="pt", sampling_rate=16000).input_values  # Batch size 1
logits = asr_model(input_values).logits.cpu().detach().numpy()[0]

In [4]:
# look at the alphabet of our model defining the labels for the logit matrix we just calculated
print(asr_processor.tokenizer.get_vocab())

{'<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3, '|': 4, 'E': 5, 'T': 6, 'A': 7, 'O': 8, 'N': 9, 'I': 10, 'H': 11, 'S': 12, 'R': 13, 'D': 14, 'L': 15, 'U': 16, 'M': 17, 'W': 18, 'C': 19, 'F': 20, 'G': 21, 'Y': 22, 'P': 23, 'B': 24, 'V': 25, 'K': 26, "'": 27, 'X': 28, 'J': 29, 'Q': 30, 'Z': 31}


The vocabulary is in a slightly unconventional shape, for example the blank ctc token is `<pad>`. Pyctcdecode tries to automatically convert that and will give a warning if it's not sure. In this case it does a good job so we don't need to modify the vocabulary by hand.

In [10]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(vocab_list)
decoder.decode(logits)

Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?


'BOIL THEM BEFORE THEY ARE PUT INTO THE SOUP OR OTHER DISH THEY MAY BE INTENDED FOR'